In [1]:


import mmcv
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import glob

from tqdm import tqdm

from glob import glob

import matplotlib.image as mpimg

import numpy as np

# decode heads (atm_head, tpn_atm_head)


import torch.nn as nn



import cv2 as cv

import math
from typing import Callable, Tuple

import torch

from scipy.ndimage import zoom

/opt/conda/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
config_file = '/project_ghent/woverbie/code/configs/config.py'
# checkpoint_file = '../../benchmarking/SegVit_test_2/models/COCOstuff_49.9.pth'
checkpoint_file = '../model/checkpoint/COCOstuff_49.9.pth'
# model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

# cpu model
model = init_segmentor(config_file, checkpoint_file, device='cpu')

# Setting the r values

# model.backbone.device = 'cpu'

r_values = np.linspace(0.995, 0.93, len(model.backbone.layers)).tolist()
for i, block in enumerate(model.backbone.layers):
    # Assuming the transformer blocks are in backbone.layers
    model.backbone.layers[i].r = r_values[i]
    # model.backbone.layers[i].device = 'cpu'


NameError: name 'init_segmentor' is not defined

In [ ]:
config_file = '/project_ghent/woverbie/code/configs/config.py'
checkpoint_file = '../../benchmarking/SegVit_test_2/models/COCOstuff_49.9.pth'
model_og = init_segmentor(config_file, checkpoint_file, device='cuda:0')

for i, block in enumerate(model_og.backbone.layers):
    model_og.backbone.layers[i].r = 1

# helper functions

In [ ]:
# imgs = sorted(glob('../dataset/camera_lidar/20180810_150607/camera/cam_front_center/*.png'))


# result = inference_segmentor(model, imgs[0])

# plt.imshow(result[0])
# plt.show()

In [ ]:
def overlap(image_rgb, result):

    base_h, base_w, _ = image_rgb.shape
    overlay_h, overlay_w = result.shape
    
    # Compute the top-left corner for centering the square overlay
    x_offset = (base_w - overlay_w) // 2
    y_offset = (base_h - overlay_h) // 2
    
    # Normalize result[0] to range from 0 to 1 for mapping to colors
    normalized_square = (result - 1) / 170  # Normalize values to range [0, 1]
    
    # Use a colormap to assign RGB colors to the normalized square
    colored_square = cv2.applyColorMap(np.uint8(normalized_square * 255), cv2.COLORMAP_JET)
    
    # Set the opacity of the overlay (0.0 means fully transparent, 1.0 means fully opaque)
    alpha = 0.5  # 50% opacity
    
    # Create a copy of the base image to preserve the original
    output_image = image_rgb.copy()
    
    # Perform the blending for each channel (R, G, B)
    for c in range(3):  # Assuming the images are RGB
        output_image[y_offset:y_offset + overlay_h, x_offset:x_offset + overlay_w, c] = \
            (alpha * colored_square[:, :, c] + (1 - alpha) * image_rgb[y_offset:y_offset + overlay_h, x_offset:x_offset + overlay_w, c])
    
    
    # Display or save the result
    plt.imshow(output_image)
    plt.show()

# Merge Function

In [ ]:
# def do_not_merge(x, mode=None):
#     return x, None, None, None

# def do_not_unmerge(x, unm_idx, src_idx, dst_idx, out_layer):
#     return x

# def custom_merge(
#     metric: torch.Tensor,
#     token_history,
#     token_history_processed,
#     r: float,
#     class_token: bool = False,
#     distill_token:bool = False
# ) -> Tuple[Callable, Callable,]:
#     # protected = 0
#     # if class_token:
#     #     protected += 1
#     # if distill_token:
#     #     protected += 1


#     # t = metric.shape[1]

#     if r >= 1:
#         return do_not_merge, do_not_unmerge
        
#     if token_history.dim() == 0:
#         return do_not_merge, do_not_unmerge

#     with torch.no_grad():
#         # metric = metric / metric.norm(dim=-1, keepdim=True)

#         # there is no history left so we can not merge yet.
#         if token_history is None:
#             return do_not_merge, do_not_unmerge
        
#         a = metric # the tokens in the current set
#         b = token_history # the history
        
#         dot_product = a @ b.transpose(-1, -2)

#         a_norm = a.norm(dim=-1, keepdim=True)  # Shape: (1, x, 1)
#         b_norm = b.norm(dim=-1, keepdim=True)  # Shape: (1, x, 1)

#         norm_product = a_norm @ b_norm.transpose(-1, -2)

#         epsilon = 1e-8
#         norm_product = norm_product + epsilon
        
#         cosine_similarity = dot_product / norm_product

#         node_max, node_idx = cosine_similarity.max(dim=-1)

#         mask = node_max > r

#         if torch.any(mask):
            
#             src_idx = torch.where(mask)[1]
#             src_idx = src_idx.unsqueeze(0).unsqueeze(-1)
            
#             dst_idx = node_idx[..., None].gather(dim=-2, index=src_idx)

#         else:
#             src_idx = None
#             dst_idx = None

#         unm_idx = torch.where(~mask)[1]
#         unm_idx = unm_idx.unsqueeze(0).unsqueeze(-1)

    

#     def merge(x: torch.Tensor, mode="mean") -> torch.Tensor:
#         # what should a merge actually do?
#         # it should return the unmatched

#         if(src_idx is not None):
#             src = x # the source of tokens is just x
    
#             n, t1, c = src.shape
    
#             _, t2, _ = unm_idx.shape
            
#             unmatched_tokens = src.gather(dim=-2, index=unm_idx.expand(n, t2, c))
#         else:
#             unmatched_tokens = x
#         return unmatched_tokens, unm_idx, src_idx, dst_idx

#     def unmerge(x: torch.Tensor, unm_idx: torch.Tensor, src_idx: torch.Tensor, dst_idx: torch.Tensor, out_layer):
#         if src_idx is not None:

            
#             # `unm` is the original tensor x
#             unm = x
#             # `dst` is the tensor containing tokens to be inserted, taken from token_history
#             dst = token_history_processed[out_layer]
        
#             # Shape parameters
#             n, unm_len, c = unm.shape  # Batch size, length of unm, channels
#             r = dst_idx.size(1)        # Number of tokens to add from dst
#             total_len = unm_len + r    # Combined length of output
        
#             # Gather the src tokens from `dst` at positions specified by `dst_idx`
#             src = torch.index_select(dst, dim=1, index=dst_idx.view(-1)).view(n, r, c)
        
#             # Concatenate `unm` and `src` along the length dimension and initialize output
#             combined = torch.cat((unm, src), dim=1)
        
#             # Create index tensor for scatter operation
#             indices = torch.cat((unm_idx, src_idx), dim=1).expand(n, total_len, c)
        
#             # Use advanced indexing to rearrange the combined tensor
#             out = torch.zeros_like(combined, device=x.device).scatter_(
#                 dim=1, index=indices, src=combined
#             )
        
#             return out
#         else:
#             return x

#     return merge, unmerge

# Custom Transformer Block

In [ ]:
# # Define a custom transformer block that includes the new layer

# class CustomTransformerBlock(nn.Module):
#     def __init__(self, original_block, layer):
#         super(CustomTransformerBlock, self).__init__()

#         self.attn = original_block.attn
#         self.norm1 = original_block.norm1
#         self.ffn = original_block.ffn
#         self.norm2 = original_block.norm2
#         self.with_cp = original_block.with_cp
#         self.layer = layer

#         self.unmerge_fn = None
#         self.unm_idx = None
#         self.src_idx = None
#         self.dst_idx = None

#         self.token_history = []

#         self.token_history_processed = {}

#         self.token_history_tensor = torch.zeros([]).to('cuda:0')
#         self.token_history_count_tensor = []

        

#         self.token_history_processed_circular = {}
#         self.token_history_circular = torch.zeros((1,2048, 1024), device='cuda:0')
#         self.max_len = 2048
#         self.token_size = 1024
#         self.index_token_history = 0
#         self.index_token_history_processed = {}

#         self.r = 0.95

#         for out_layer in [7, 15, 23]:
#             self.token_history_processed.setdefault(out_layer, torch.zeros((1,2048, 1024), device='cuda:0'))
#             self.index_token_history_processed[out_layer] = 0

#     def forward(self, x):

#         def _inner_forward(x):
#             # self attention layer
#             x = self.attn(self.norm1(x), identity=x)

#             # if layer is one that should be reduced
#             if self.layer % 3 == 0:
#                 _, n_tokens, _ = x.shape

#                 # if len(self.token_history_count_tensor)>0:
#                 # stacked_history = 
#                 # stacked_processed_history = 

#                 merge, unmerge = custom_merge(x, self.token_history_circular , self.token_history_processed, r=self.r)
                
#                 x, size, unm_idx, src_idx, dst_idx = merge_wavg(merge, x)

#                 self.unmerge_fn = unmerge
#                 self.unm_idx = unm_idx
#                 self.src_idx = src_idx
#                 self.dst_idx = dst_idx
                
#                 _, batch_size, token_size = x.shape
#                 end_idx = self.index_token_history + batch_size
#                 if end_idx <= self.max_len:
#                     # Add tokens without wrapping
#                     self.token_history_circular[0, self.index_token_history:end_idx, :] = x[0]
#                 else:
#                     # Wrap around: split the tokens and write in two parts
#                     overflow = end_idx - self.max_len
#                     self.token_history_circular[0, self.index_token_history:self.max_len, :] = x[0, :batch_size - overflow, :]
#                     self.token_history_circular[0, :overflow, :] = x[0, batch_size - overflow:, :]
        
#                 # Update the index, wrapping around
#                 self.index_token_history = end_idx % self.max_len

#             # I add myself to the token merge
#             x = self.ffn(self.norm2(x), identity=x)

#             return x

#         if self.with_cp and x.requires_grad:

#             x = cp.checkpoint(_inner_forward, x)
#         else:
#             x = _inner_forward(x)

#         return x

#     def unmerge(self, x, unm_idx, src_idx, dst_idx, out_layer):

#         if self.layer % 3 == 0 or unm_idx is not None:

#             reconstructed_tokens = self.unmerge_fn(x, unm_idx, src_idx, dst_idx, out_layer)

#             _, batch_size, token_size = x.shape
#             end_idx = self.index_token_history_processed[out_layer] + batch_size
            
#             if end_idx <= self.max_len:
#                 # Add tokens without wrapping
#                 self.token_history_processed[out_layer][0, self.index_token_history_processed[out_layer]:end_idx, :] = x[0]
#             else:
#                 # Wrap around: split the tokens and write in two parts
#                 overflow = end_idx - self.max_len
#                 self.token_history_processed[out_layer][0, self.index_token_history_processed[out_layer]:self.max_len, :] = x[0, :batch_size - overflow, :]
#                 self.token_history_processed[out_layer][0, :overflow, :] = x[0, batch_size - overflow:, :]

#             # Update the index, wrapping around if needed
#             self.index_token_history_processed[out_layer] = end_idx % self.max_len

#             return reconstructed_tokens
#         else:
#             return x
        




# Inference

In [ ]:
class_map = {
    (1,2,3,4,15,16,17, 47): 2, # car 1
    # 2: 2, # car 2
    # 3: 2, # car 3
    # 4: 2, # car 4
    (5,6,7,8): 1, # bicycle 1
    # 6: 1, # bicycle 2
    # 7: 1, # bicycle 3
    # 8: 1, # bicycle 4
    (9,10,11): 0, # pedestrian 1
    # 10: 0, # pedestri3n 2
    # 11: 0, # pedestrian 3
    (12,13,14,32): 7, # Truck 1
    # 13: 7, # Truck 2
    # 14: 7, # Truck 3
    # (15,16,17): 2, #Small vehicles 1"
    # 16: 2, #Small vehicles 2"
    # 17: 2, #Small vehicles 3"
    (18,19,20): 9, #Traffic signal 1"
    # 19: 9, #Traffic signal 2"
    # 20: 9, #Traffic signal 3"
    (21,22,23): 11,#Traffic sign 1"
    # 22: 11,#Traffic sign 2"
    # 23: 11, #Traffic sign 3"
    (24,25): 7, #Utility vehicle 1"
    # 25: 7 ,#Utility vehicle 2"
    26: 134 ,#Sidebars"
    27: -1,#Speed bumper"
    28: -1,#Curbstone"
    # 29: 137,#Solid line"
    30: 9,#Irrelevant signs"
    31: -1,#Road blocks"
    # 32: 7, #Tractor"
    # 33: 137 ,#Non-drivable street"
    # 34: 137 ,#Zebra crossing"
    35: -1,#Obstacles / trash"
    36: -1,#Poles"
    37: -1,#RD restricted area"
    38: (14,15,16,17,18,19,20,21,22,23),  #Animals"
    39: -1, #Grid structure"
    40: -1, #Signal corpus"
    # 41: 137 ,#Drivable cobblestone"
    42: -1 ,#Electronic traffic" (digital board contiaining info
    # 43: 137 ,#Slow drive area"
    44: (82, 85, 130, 157),#Nature object"
    45: -1, #Parking area"
    # 46: 137,#Sidewalk":
    # 47: 137 ,#Ego car"
    # 48: 137 ,#Painted driv. instr."
    49: -1 ,#Traffic guide obj."
    # 50: 137 ,#Dashed line"
    # 51: 137,#RD normal street"
    (29,33,34,41,43,46,48,50,51): 137,
    52: (145, 94), #Sky"
    53: (84 ,116, 168, 169, 159, 160, 161, 162, 163, 164, 165), #Buildings"
    54: -1,#Blurred area"
    55: 99,#Rain dirt"
}

In [ ]:
def similarity_percentage(array1, array2):
    # Ensure the arrays have the same shape
    if array1.shape != array2.shape:
        raise ValueError("Arrays must have the same shape for comparison.")
    
    # Calculate similarity
    identical_elements = np.sum(array1 == array2)
    total_elements = array1.size
    similarity = (identical_elements / total_elements) * 100
    
    return similarity

In [ ]:
def normalize_gt(gt, map):

    combined_mask = np.zeros(gt.shape[:2], dtype=float)

    for gt_id, pred_id in map.items():
        if isinstance(gt_id, tuple):
            mask = np.zeros(gt.shape[:2], dtype=float)

            for ground_truth in gt_id:
                mask += np.isin(gt, ground_truth)  
        else:
            mask = np.isin(gt, gt_id)

        if isinstance(pred_id, tuple):
            combined_mask += mask * pred_id[0]
        else:
            combined_mask += mask * pred_id

    cut = int((1920 - 1208) / 2)

    test = combined_mask[:, cut:1920-cut]

    return test

def normalize_result(result, map):
    combined_mask = np.zeros(result.shape[:2], dtype=float)
    
    for _ , cls in map.items():
        mask = np.isin(result, cls)
        
        if isinstance(cls, tuple):
            combined_mask += mask * cls[0]
        else:
            combined_mask += mask * cls
    
    return combined_mask

In [ ]:
# config_file = '/project_ghent/woverbie/code/configs/config.py'
# checkpoint_file = '../../benchmarking/SegVit_test_2/models/COCOstuff_49.9.pth'
# model_og = init_segmentor(config_file, checkpoint_file, device='cuda:0')

# for i, block in enumerate(model_og.backbone.layers):
#     # Assuming the transformer blocks are in backbone.layers
#     model_og.backbone.layers[i] = CustomTransformerBlock(block, i)
#     model_og.backbone.layers[i].r = 1
#     # print(model_og.backbone.layers[i].r)

In [ ]:
## Test with an inference function
import cv2

imgs = sorted(glob('../dataset/camera_lidar/20180810_150607/camera/cam_front_center/*.png'))

labels = sorted(glob("../dataset/labels/20181204_135952/*"))

perc_list = []

prunes = []

reduced_tokens_heatmap = None

# Display the image
for i, (img_path, label_path) in enumerate(zip(imgs, labels)):
    

    print('image', i)
    
    result = inference_segmentor(model, img_path)
    result_og = inference_segmentor(model_og, img_path)

    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # perc = similarity_percentage(gt_norm, res_norm)

    # perc_list.append(perc)

    # overlap(image_rgb, result[0])
    # overlap(image_rgb, result_og[0])

    result1 = result[0] - result_og[0]
    binary_result = (result1 != 0).astype(int)
    
    # overlap(image_rgb, binary_result)
    percentage_ones = (binary_result.sum() / binary_result.size) * 100
    perc_list.append(percentage_ones)

    reduction = []

    for i, layer in enumerate(model.backbone.layers):
        # print(i)
        
        
        if layer.unm_idx is not None:
            
            # print('unm', layer.unm_idx.shape[1])
    
            if layer.src_idx is not None:
                # print('src', layer.src_idx.shape[1])
                reduction.append(layer.src_idx.shape[1])

            else:
                reduction.append(0)

    prunes.append(reduction)

    pruned_tokens = []
    for i, layer in reversed(list(enumerate(model.backbone.layers))):
        if layer.unm_idx is not None:
            if len(pruned_tokens) == 0:
                pruned_tokens = [0] * len(layer.unm_idx[0])

            if layer.src_idx is not None:
                # print(layer.src_idx)
                temp = pruned_tokens

                pruned_tokens = [0] * ( len(layer.unm_idx[0]) + len(layer.src_idx[0]))


                for i, unm_idx in enumerate(layer.unm_idx[0]):
                    # print('a', unm_idx)
                    pruned_tokens[unm_idx] = temp[i]

                for i, src_idx in enumerate(layer.src_idx[0]):
                    pruned_tokens[src_idx] = 1

    #     print(len(pruned_tokens))

    # print(pruned_tokens)

    if len(pruned_tokens) == 0:
        pruned_tokens = [0]*1024
        
    tokens_reduced = np.array(pruned_tokens).reshape((32, 32))
    
    # scaled_tokens_reduced = np.repeat(np.repeat(tokens_reduced, 16, axis=0), 16, axis=1)
    zoom_factor = 1208 / 32
    scaled_tokens_reduced = zoom(tokens_reduced, zoom=(zoom_factor, zoom_factor), order=1)
    # print('tokens_reduced', pruned_tokens.count(1))
    # plt.imshow(tokens_reduced)
    # plt.show()

    if reduced_tokens_heatmap is not None:
        reduced_tokens_heatmap += scaled_tokens_reduced
    else:
        reduced_tokens_heatmap = scaled_tokens_reduced

    # overlap(image_rgb, scaled_tokens_reduced)



encode_times_np = np.array(model.encode_times)
np.save("encode_times_cpu.npy", encode_times_np)

# eval

In [ ]:
# model.backbone.layers[0].token_history_processed

In [ ]:
num_bins = 100

# Plot the histogram
plt.figure(figsize=(10, 6))
plt.hist(perc_list, bins=num_bins, edgecolor='black', alpha=0.75)

# Formatting the plot
plt.title("loss between no reduction  and reduction (in percentage)", fontsize=16)
plt.xlabel("time", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xlim((0,50))

# Show the plot
plt.show()


perc_np = np.array(perc_list)

np.save("perc_loss_cpu.npy", perc_np)

In [ ]:
model.encode_times

In [ ]:
num_bins = 1000

# Plot the histogram
plt.figure(figsize=(10, 6))
# plt.hist(model.backbone.reconstruction_times, bins=num_bins, edgecolor='red', alpha=0.75)
plt.hist(model.encode_times, bins=num_bins, edgecolor='black', alpha=0.75)
# plt.hist(model_og.encode_times, bins=num_bins,color='red', edgecolor='red', alpha=0.75)

# Formatting the plot
plt.title("Models encode times Distribution of Values", fontsize=16)
plt.xlabel("time", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.xlim((0.00,0.12))

# Show the plot
plt.show()



In [ ]:
sums = np.sum(prunes, axis=1)

# Plot the sums as a line chart
plt.figure(figsize=(10, 6))
plt.plot(sums, label='Sum over 2nd dimension')
# plt.xlabel('Index of the 1st Dimension')
# plt.ylabel('Sum of the 2nd Dimension')
plt.title('Change in reduction Over the 2nd Dimension')
plt.legend()
plt.grid()
plt.show()



tokens_pruned_np = np.array(sums)

np.save("tokens_pruned_cpu.npy", tokens_pruned_np)

In [ ]:
window_size = 30
moving_avg = [
    np.mean(sums[max(0, i - window_size + 1):i + 1]) 
    for i in range(len(sums))
]

# Plot the moving average as a line chart
plt.figure(figsize=(10, 6))
plt.plot(moving_avg, label='Moving Average (window size = 30)')
plt.title('Change in Reduction Over the 2nd Dimension')
plt.legend()
plt.grid()
plt.show()

In [ ]:
column_averages = np.mean(prunes, axis=0)

plt.subplot(1, 2, 2)
plt.plot(column_averages, marker='o', label='Column Averages')
plt.title('reductions per layer')
plt.legend()
plt.grid()

reduction_per_layer_np = np.array(column_averages)

np.save("reductions_per_layer_cpu.npy", reduction_per_layer_np)

In [ ]:
num_bins = 1000

# Plot the histogram
plt.figure(figsize=(10, 6))
# plt.hist(s, bins=num_bins, edgecolor='black', alpha=0.75)
plt.hist(model.decode_times, bins=num_bins, edgecolor='red', alpha=0.3)

# Formatting the plot
plt.title("Models decode times Distribution of Values", fontsize=16)
plt.xlabel("time", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.xlim((0.02,0.06))

# Show the plot
plt.show()

In [ ]:
# Heatmap of tokens reduced

# sum_ones = np.sum(reduced_tokens_heatmap, axis=0)

# Step 2: Normalize by the total number of frames
# num_frames = reduced_tokens_heatmap.shape[0]
# heatmap = sum_ones / num_frames  # This gives a 2D array with values between 0 and 1

# Step 3: Visualize as a heatmap
plt.figure(figsize=(8, 6))
plt.imshow(reduced_tokens_heatmap, cmap='hot', interpolation='nearest')
plt.colorbar(label='Frequency of reduction')
plt.title('Frequency Heatmap of reduced tokens')
plt.xlabel('Width')
plt.ylabel('Height')
plt.show()

reduced_tokens_heatmap_np = np.array(reduced_tokens_heatmap)

np.save("reduced_tokens_heatmap_np_cpu.npy", reduced_tokens_heatmap_np)

# flops calculator

In [ ]:
config_file = '/project_ghent/woverbie/code/configs/config.py'
checkpoint_file = '../../benchmarking/SegVit_test_2/models/COCOstuff_49.9.pth'
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

# Setting the r values

r_values = np.linspace(0.995, 0.93, len(model.backbone.layers)).tolist()
for i, block in enumerate(model.backbone.layers):
    # Assuming the transformer blocks are in backbone.layers
    model.backbone.layers[i].r = r_values[i]


In [ ]:

## Test with an inference function
import cv2
import torch
from torch.profiler import profile, ProfilerActivity
from mmseg.datasets.pipelines import Compose

imgs = sorted(glob('../dataset/camera_lidar/20180810_150607/camera/cam_front_center/*.png'))

cfg = model.cfg  # The config file used to initialize the model
test_pipeline = Compose(cfg.data.test.pipeline)

perc_list = []

prunes = []

reduced_tokens_heatmap = None




def profile_inference(model, img):
    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], 
                 with_flops=True,  # Enable FLOP counting
                 record_shapes=True) as prof:
        # Perform inference
        _ = inference_segmentor(model, img)

    # Process and summarize the profiling data
    # flops = prof.key_averages().table(sort_by="cuda_time_total", row_limit=10)
    # print(prof.key_averages().table(sort_by="flops", row_limit=10))
    return prof


prof_list = []

# Display the image
for i, (img_path, label_path) in enumerate(zip(imgs, labels)):

    prof = profile_inference(model, img_path)
    
    avg = sum(op.flops for op in prof.key_averages() if hasattr(op, 'flops'))

    print(avg)

    prof_list.append(avg)
        

In [ ]:
# Plot the sums as a line chart
plt.figure(figsize=(10, 6))
plt.plot(prof_list, label='flops')
# plt.xlabel('Index of the 1st Dimension')
# plt.ylabel('Sum of the 2nd Dimension')
plt.title('flops')
plt.legend()
plt.grid()
plt.show()

# perc_np = np.array(prof_list)

np.save("prof_list.npy", prof_list)

# imports form numpy

In [ ]:
prof_list = np.load('./prof_list.npy')
tokens_pruned = np.load('./tokens_pruned.npy')

In [ ]:
prof_list

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example data
# tokens_pruned = np.array([0, 256, 512, 768, 1024])  # Max = 1024
# prof_list = np.array([0, 195955603867, 391911207734, 587866811601, 783822415468])  # Max = 783822415468

window_size = 2
moving_avg_tokens = [
    np.mean(tokens_pruned[max(0, i - window_size + 1):i + 1]) 
    for i in range(len(tokens_pruned))
]

moving_avg_prof = [
    np.mean(prof_list[max(0, i - window_size + 1):i + 1]) 
    for i in range(len(prof_list))
]

# Create the figure and first axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot tokens_pruned on the first axis
ax1.plot(moving_avg_tokens, label="Tokens Pruned", color="blue")
ax1.set_xlabel("Index")
ax1.set_ylabel("Tokens Pruned", color="blue")
ax1.tick_params(axis="y", labelcolor="blue")
ax1.set_ylim(0, 1024)

# Create the second axis
ax2 = ax1.twinx()

# Plot prof_list on the second axis
ax2.plot(moving_avg_prof, label="Prof List", color="orange")
ax2.set_ylabel("Prof List", color="orange")
ax2.tick_params(axis="y", labelcolor="orange")
ax2.set_ylim(0, 783822415468)

# Add a title
plt.title("Overlay of Two Data Sources with Twin Axes")

# Add a grid for clarity
ax1.grid(True, which='both', linestyle='--', linewidth=0.5)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
if torch.cuda.is_available():
    print(f"CUDA is available. Current device: {torch.cuda.current_device()}")
    print(f"Model is on device: {next(model_og.parameters()).device}")
else:
    print("CUDA is not available. The model is likely on CPU.")

In [ ]:
num_bins = 100

encode_times_1 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_1.0.npy')
encode_times_98 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_0.98.npy')

# Plot the histogram
plt.figure(figsize=(10, 6))
plt.hist(encode_times_1, bins=num_bins, edgecolor='red', alpha=0.75)
plt.hist(encode_times_98, bins=num_bins, edgecolor='black', alpha=0.75)
# plt.hist(model_og.encode_times, bins=num_bins,color='red', edgecolor='red', alpha=0.75)

# Formatting the plot
plt.title("Models encode times Distribution of Values", fontsize=16)
plt.xlabel("time", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xlim((0.00,0.15))

# Show the plot
plt.show()



In [ ]:
encode_times

In [ ]:
VIDEO_GAIMERSHEIM = sorted(glob('../dataset/camera_lidar/20180810_150607/camera/cam_front_center/*.png'))


In [ ]:
len(VIDEO_GAIMERSHEIM)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

num_bins = 1000

encode_times_1 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_1.0.npy')
encode_times_98 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_0.98.npy')
encode_times_96 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_0.96.npy')
encode_times_94 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_0.94.npy')
encode_times_92 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_0.92.npy')
encode_times_90 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_0.9.npy')
encode_times_88 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_0.88.npy')
encode_times_86 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_0.86.npy')
encode_times_84 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_0.84.npy')





# KDE for smooth line
kde_1 = gaussian_kde(encode_times_1)
kde_98 = gaussian_kde(encode_times_98)
kde_96 = gaussian_kde(encode_times_96)
kde_94 = gaussian_kde(encode_times_94)
kde_92 = gaussian_kde(encode_times_92)
kde_90 = gaussian_kde(encode_times_90)
kde_88 = gaussian_kde(encode_times_88)
kde_86 = gaussian_kde(encode_times_86)
kde_84 = gaussian_kde(encode_times_84)


# Generate a range of values for the x-axis
x = np.linspace(0.00, 0.1, 500)

# Plot the KDEs
plt.figure(figsize=(10, 6))
# plt.plot(x, kde_1(x), label="Encode Times 1.0", color='red', alpha=0.75)
plt.plot(x, kde_98(x), label="threshold = 0.98", color='black', alpha=0.75)
plt.plot(x, kde_96(x), label="threshold = 0.96", color='green', alpha=0.75)
plt.plot(x, kde_94(x), label="threshold = 0.94", color='blue', alpha=0.75)
plt.plot(x, kde_92(x), label="threshold = 0.92", color='orange', alpha=0.75)
plt.plot(x, kde_90(x), label="threshold = 0.90", color='yellow', alpha=0.75)
plt.plot(x, kde_88(x), label="threshold = 0.88", color='pink', alpha=0.75)
plt.plot(x, kde_86(x), label="threshold = 0.86", color='gray', alpha=0.75)
plt.plot(x, kde_84(x), label="threshold = 0.84", color='purple', alpha=0.75)

# Formatting the plot
plt.title("Models Encode Times Distribution of Values", fontsize=16)
plt.xlabel("Time", fontsize=14)
plt.ylabel("Density", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.xlim((0.00, 0.15))
plt.legend()

# Show the plot
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load the datasets
thresholds = [0.98, 0.96, 0.94, 0.92, 0.90, 0.88, 0.86, 0.84]
encode_times = {
    threshold: np.load(f'/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_{threshold}.npy')
    for threshold in thresholds
}

# Calculate mean and median for each threshold
means = [np.mean(encode_times[threshold]) for threshold in thresholds]
medians = [np.median(encode_times[threshold]) for threshold in thresholds]

# Plot mean and median
plt.figure(figsize=(10, 6))
plt.plot(thresholds, means, label="Mean Encode Time", marker='o', linestyle='-', color='blue')
plt.plot(thresholds, medians, label="Median Encode Time", marker='s', linestyle='--', color='orange')

# Formatting the plot
plt.title("Mean and Median Encode Times for Different Thresholds", fontsize=16)
plt.xlabel("Threshold", fontsize=14)
plt.ylabel("Encode Time (s)", fontsize=14)
plt.ylim(0,0.06)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.xticks(thresholds)  # Ensure all thresholds are labeled on the x-axis

# Show the plot
plt.show()


In [ ]:
len(encode_times_84)


In [ ]:
len(encode_times_96)

In [ ]:
np.median(encode_times_84)

In [ ]:
reduced_token_heatmap = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/reduced_tokens_heatmap_lin_0.995_0.93.npy')

# reduced_token_heatmap = reduced_token_heatmap / len(encode_times_98)

plt.figure(figsize=(5, 4))
plt.imshow(reduced_token_heatmap, cmap='hot', interpolation='nearest', vmin=0, vmax=len(encode_times_98))
# plt.colorbar(label='#Frames where token is reduced')
plt.title('Frequency Heatmap of Reduced Tokens \n with a Linear Threshold')
plt.xlabel('Width')
plt.ylabel('Height')
plt.show()

reduced_token_heatmap = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/reduced_tokens_heatmap_all_layers_0.98.npy')

# reduced_token_heatmap = reduced_token_heatmap / len(encode_times_98)

plt.figure(figsize=(5, 4))
plt.imshow(reduced_token_heatmap, cmap='hot', interpolation='nearest', vmin=0, vmax=len(encode_times_98))
# plt.colorbar(label='#Frames where token is reduced')
plt.title('Frequency Heatmap of Reduced Tokens \n with a Fixed Threshold = 0.98')
plt.xlabel('Width')
plt.ylabel('Height')
plt.show()

reduced_token_heatmap = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/reduced_tokens_heatmap_all_layers_0.92.npy')

# reduced_token_heatmap = reduced_token_heatmap / len(encode_times_98)

plt.figure(figsize=(5, 4))
plt.imshow(reduced_token_heatmap, cmap='hot', interpolation='nearest', vmin=0, vmax=len(encode_times_98))
plt.colorbar(label='#Frames where token is reduced')
plt.title('Frequency Heatmap of Reduced \n Tokens with a Fixed Threshold = 0.9')
plt.xlabel('Width')
plt.ylabel('Height')
plt.show()

In [ ]:
# acc_98 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/pixel_wise_acc_list_0.98.npy')
# acc_96 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/pixel_wise_acc_list_0.96.npy')
# acc_94 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/pixel_wise_acc_list_0.94.npy')
# acc_92 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/pixel_wise_acc_list_0.92.npy')
# acc_90 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/pixel_wise_acc_list_0.9.npy')
# acc_88 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/pixel_wise_acc_list_0.88.npy')
# acc_86 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/pixel_wise_acc_list_0.86.npy')
# acc_84 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/pixel_wise_acc_list_0.84.npy')

# # KDE for smooth line
# # kde_1 = gaussian_kde(acc_1)
# kde_98 = gaussian_kde(acc_98)
# kde_96 = gaussian_kde(acc_96)
# kde_94 = gaussian_kde(acc_94)
# kde_92 = gaussian_kde(acc_92)
# kde_90 = gaussian_kde(acc_90)
# kde_88 = gaussian_kde(acc_88)
# kde_86 = gaussian_kde(acc_86)
# kde_84 = gaussian_kde(acc_84)

# # Generate a range of values for the x-axis
# x = np.linspace(0, 100, 500)

# # Plot the KDEs
# plt.figure(figsize=(10, 6))

# plt.plot(x, kde_98(x), label="Loss r=0.98", color='black', alpha=0.75)
# plt.plot(x, kde_96(x), label="Loss r=0.96", color='green', alpha=0.75)
# plt.plot(x, kde_94(x), label="Loss r=0.94", color='blue', alpha=0.75)
# plt.plot(x, kde_92(x), label="Loss r=0.92", color='orange', alpha=0.75)
# plt.plot(x, kde_90(x), label="Loss r=0.90", color='red', alpha=0.75)
# plt.plot(x, kde_88(x), label="Loss r=0.88", color='pink', alpha=0.75)
# plt.plot(x, kde_86(x), label="Loss r=0.86", color='gray', alpha=0.75)
# plt.plot(x, kde_84(x), label="Loss r=0.84", color='purple', alpha=0.75)



# # Formatting the plot
# plt.title("Pixel-wise Loss compared to original model", fontsize=16)
# plt.xlabel("Loss (in %)", fontsize=14)
# plt.ylabel("Percentage of processed frames", fontsize=14)
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.legend()

# # Show the plot
# plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt




In [ ]:
np.median(acc_84)

In [ ]:
print("98", round(np.mean(acc_98),2), round(np.median(acc_98),2))
print("96", round(np.mean(acc_96),2), round(np.median(acc_96),2))
print("94", round(np.mean(acc_94),2), round(np.median(acc_94),2))
print("92", round(np.mean(acc_92),2), round(np.median(acc_92),2))
print("90", round(np.mean(acc_90),2), round(np.median(acc_90),2))
print("88", round(np.mean(acc_88),2), round(np.median(acc_88),2))
print("86", round(np.mean(acc_86),2), round(np.median(acc_86),2))
print("84", round(np.mean(acc_84),2), round(np.median(acc_84),2))

In [ ]:

# num_bins = 1000

# tokens_pruned_98 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/tokens_pruned_0.98.npy')
# tokens_pruned_98 = np.sum(tokens_pruned_98, axis=1)

# # Plot the sums as a line chart
# plt.figure(figsize=(10, 6))
# plt.plot(tokens_pruned_98, label='# Reduced tokens for frame')
# plt.title('threshold = 0.98',  fontsize=25)
# plt.legend()
# plt.grid()
# plt.show()

# tokens_pruned_92 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/tokens_pruned_0.92.npy')
# tokens_pruned_92 = np.sum(tokens_pruned_92, axis=1)

# # Plot the sums as a line chart
# plt.figure(figsize=(10, 6))
# plt.plot(tokens_pruned_92, label='Sum over 2nd dimension')
# plt.title('threshold = 0.92',  fontsize=25)
# plt.legend()
# plt.grid()
# plt.show()


# tokens_pruned_84 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/tokens_pruned_0.84.npy')
# tokens_pruned_84 = np.sum(tokens_pruned_84, axis=1)

# # Plot the sums as a line chart
# plt.figure(figsize=(10, 6))
# plt.plot(tokens_pruned_84, label='Sum over 2nd dimension')
# # plt.xlabel('Index of the 1st Dimension')
# # plt.ylabel('Sum of the 2nd Dimension')
# plt.title('threshold = 0.84',  fontsize=25)
# plt.legend()
# plt.grid()
# plt.show()


# plt.figure(figsize=(10, 6))

# plt.plot(tokens_pruned_84, label='Threshold = 0.84', alpha=0.75)
# plt.plot(tokens_pruned_92, label='Threshold = 0.92',alpha=0.75)
# plt.plot(tokens_pruned_98, label='Threshold = 0.98', alpha=0.75)
# plt.plot([1024] * len(tokens_pruned_84), label='Max tokens to reduce', alpha=1)
# plt.title('Reduced Tokens through Video Frames')
# plt.xlabel("Frame index", fontsize=14)
# plt.ylabel("# Reduced Tokens", fontsize=14)
# plt.legend()
# plt.grid()
# plt.show()



In [ ]:
len(tokens_pruned_98)

# trying to get where it sometimes is very high

In [ ]:
tokens_pruned_96

In [ ]:
test = tokens_pruned_98[13000:15688]

In [ ]:
tokens_pruned_98 = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/tokens_pruned_0.98.npy')
tokens_pruned_98 = np.sum(tokens_pruned_98, axis=1)

# Plot the sums as a line chart
plt.figure(figsize=(10, 6))
plt.plot(tokens_pruned_98, label='# Reduced tokens for frame')
plt.title('Change in Reduction of Tokens through Frames')
plt.legend()
plt.xlabel("Frames", fontsize=14)
plt.ylabel("# reduced tokens", fontsize=14)
plt.grid()
plt.show()


plt.figure(figsize=(4, 4))
plt.plot(tokens_pruned_98, label='# Reduced tokens for frame')
plt.title('Change in Reduction of Tokens through Frames')
plt.xlim(15000,15500)
plt.ylim(0,1030)
plt.legend()
plt.xlabel("Frames", fontsize=14)
plt.ylabel("# reduced tokens", fontsize=14)
plt.grid()
plt.show()

In [ ]:
for i, img_path in enumerate(VIDEO_GAIMERSHEIM):
    if i % 30 == 0 and i>3000:
        img = mpimg.imread(img_path)
        plt.imshow(img)
        plt.title(f"frame {i}")
        plt.show()
    
    

In [ ]:
tokens_pruned = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/tokens_pruned_lin_0.995_0.93.npy')


column_averages = np.mean(tokens_pruned, axis=0)

plt.subplot(1, 2, 2)
plt.plot(column_averages, marker='o', label='Column Averages')
plt.title('reductions per layer')
plt.legend()
plt.grid()

reduction_per_layer_np = np.array(column_averages)

np.save("reductions_per_layer_cpu.npy", reduction_per_layer_np)

# The OG model

In [ ]:
config_file = '/project_ghent/woverbie/code/SegVit/configs/segvit/segvit_vit-l_jax_512x512_80k_cocostuff10k.py'
checkpoint_file = '../../benchmarking/SegVit_test_2/models/COCOstuff_49.9.pth'
model_og = init_segmentor(config_file, checkpoint_file, device='cpu')

In [ ]:
imgs = sorted(glob('../dataset/camera_lidar/20180810_150607/camera/cam_front_center/*.png'))
imgs = imgs[0:3]

for img in imgs:
    result = inference_segmentor(model_og, img)


In [ ]:
model_og.encode_times

# different reduction intervals

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load the datasets
thresholds = [1,2,3,4,6,8,12,24]
encode_times = {
    threshold: np.load(f'/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_lin_int_{threshold}.npy')
    for threshold in thresholds
}

# Prepare the data for histogram line plots
labels = [f"Reduction Interval = {threshold}" for threshold in thresholds]
colors = ['black', 'green', 'blue', 'orange', 'red', 'pink', 'gray', 'purple']
datasets = [encode_times[threshold] for threshold in thresholds]

# Set up the plot
plt.figure(figsize=(8, 4))

# Generate and plot histogram data as lines
bins = np.linspace(0, 0.1, 50)  # Define bins (same for all datasets)
for data, label, color in zip(datasets, labels, colors):
    counts, bin_edges = np.histogram(data, bins=bins)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # Compute bin centers
    plt.plot(bin_centers, counts, label=label, color=color, alpha=0.75)

# Formatting the plot
plt.title("Encode Times Distribution", fontsize=16)
plt.xlabel("Encode Time (s)", fontsize=14)
plt.ylabel("# Frames", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend()

# Show the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load the datasets
thresholds = [1,2,3,4,6,8,12,24]
encode_times = {
    threshold: np.load(f'/project_ghent/woverbie/code/notebooks/benchmarking/encode_times_lin_int_{threshold}.npy')
    for threshold in thresholds
}

# Calculate mean and median for each threshold
means = [np.mean(encode_times[threshold]) for threshold in thresholds]
medians = [np.median(encode_times[threshold]) for threshold in thresholds]

# Plot mean and median
plt.figure(figsize=(4, 4))
plt.plot(thresholds, means, label="Mean Encode Time", marker='o', linestyle='-', color='blue')
plt.plot(thresholds, medians, label="Median Encode Time", marker='s', linestyle='--', color='orange')

# Formatting the plot
plt.title("Mean and Median Encode Times \n for Different Thresholds", fontsize=16)
plt.xlabel("Threshold", fontsize=14)
plt.ylabel("Encode Time (s)", fontsize=14)
plt.ylim(0,0.06)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.xticks(thresholds)  # Ensure all thresholds are labeled on the x-axis

# Show the plot
plt.show()


In [ ]:
def calc_metrics(arr):
    mean = np.mean(arr)
    median = np.median(arr)
    std_dev = np.std(arr)
    variance = np.var(arr)
    min_val = np.min(arr)
    max_val = np.max(arr)
    range_val = np.ptp(arr)  # Range (max - min)
    sum_val = np.sum(arr)
    count = len(arr)  # Number of elements
    
    # Display results
    print(f"Mean: {mean}")
    print(f"Median: {median}")
    print(f"Standard Deviation: {std_dev}")
    print(f"Variance: {variance}")
    print(f"Minimum: {min_val}")
    print(f"Maximum: {max_val}")
    print(f"Range: {range_val}")
    print(f"Sum: {sum_val}")
    print(f"Count: {count}")



In [ ]:
calc_metrics(encode_times_8)

In [ ]:
calc_metrics(encode_times_24)

In [ ]:
calc_metrics(encode_times_6)

In [ ]:
tokens_pruned = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/tokens_pruned_lin_int_1.npy')


column_averages = np.mean(tokens_pruned, axis=0)
column_median = np.median(tokens_pruned, axis=0)

# plt.subplot(1, 2, 2)
plt.figure(figsize=(8, 3))

plt.plot(column_median, label="Median Reduction of Tokens in Layer", marker='s', linestyle='--', color='orange')
plt.plot(column_averages, marker='o', label='Mean Reduction of Tokens in Layer')
plt.title('Token Reduction per Layer')
plt.xlabel('Layers in Vision Transformer Encoder')
plt.ylabel('# Reduced Tokens in Layer')
plt.legend()

plt.grid()
plt.show()

# reduction_per_layer_np = np.array(column_averages)

# np.save("reductions_per_layer_cpu.npy", reduction_per_layer_np)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load the datasets
thresholds = [1,2,3,4,6,8,12,24]
encode_times = {
    threshold: np.load(f'/project_ghent/woverbie/code/notebooks/benchmarking/pixel_wise_acc_list_lin_int_{threshold}.npy')
    for threshold in thresholds
}

# Calculate mean and median for each threshold
means = [np.mean(encode_times[threshold]) for threshold in thresholds]
medians = [np.median(encode_times[threshold]) for threshold in thresholds]

# Plot mean and median
plt.figure(figsize=(4,4))
plt.plot(thresholds, means, label="Mean Encode Time", marker='o', linestyle='-', color='blue')
plt.plot(thresholds, medians, label="Median Encode Time", marker='s', linestyle='--', color='orange')

# Formatting the plot
plt.title("Mean and Median Pixel-Wise Loss \n for Different Reduction Intervals", fontsize=16)
plt.xlabel("Reduction Interval", fontsize=14)
plt.ylabel("Pixel-Wise Loss (%)", fontsize=14)
plt.ylim(0,10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend()
plt.xticks(thresholds)  # Ensure all thresholds are labeled on the x-axis

# Show the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load the datasets
thresholds = [1,2,3,4,6,8,12,24]
encode_times = {
    threshold: np.load(f'/project_ghent/woverbie/code/notebooks/benchmarking/pixel_wise_acc_list_lin_int_{threshold}.npy')
    for threshold in thresholds
}

# Prepare the data for histogram line plots
labels = [f"Reduction Interval = {threshold}" for threshold in thresholds]
colors = ['black', 'green', 'blue', 'orange', 'red', 'pink', 'gray', 'purple']
datasets = [encode_times[threshold] for threshold in thresholds]

# Set up the plot
plt.figure(figsize=(8, 4))

# Generate and plot histogram data as lines
bins = np.linspace(0, 20, 100)  # Define bins (same for all datasets)
for data, label, color in zip(datasets, labels, colors):
    counts, bin_edges = np.histogram(data, bins=bins)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # Compute bin centers
    plt.plot(bin_centers, counts, label=label, color=color, alpha=0.75)

# Formatting the plot
plt.title("Pixel-Wise Loss Distribution compared to Original Model \n for Different Reduction Intervals", fontsize=16)
plt.xlabel("Pixel-Wise Loss (%)", fontsize=14)
plt.ylabel("# Frames", fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend()

# Show the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

num_bins = 1000

encode_times = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/model_og_encode_times_gpu.npy')
decode_times = np.load('/project_ghent/woverbie/code/notebooks/benchmarking/model_og_decode_times_gpu.npy')

# project_ghent/woverbie/code/notebooks/benchmarking/encode_times_lin_int_1.npy
# project_ghent/woverbie/code/notebooks/benchmarking/pixel_wise_acc_list_int_0.995_0.93.npy

# KDE for smooth line
kde_1 = gaussian_kde(encode_times)
kde_2 = gaussian_kde(decode_times)


# Generate a range of values for the x-axis
x = np.linspace(0.00, 0.08, 500)

# Plot the KDEs
plt.figure(figsize=(6, 4))

plt.plot(x, kde_1(x), label="Encode Times", color='red', alpha=0.75)
plt.plot(x, kde_2(x), label="Decode Times", color='green', alpha=0.75)

# Formatting the plot
plt.title("Distribution of the Encode and Decode\n Times during Inference of the Original Model")
plt.xlabel("Time (s)")
plt.ylabel("# Frames")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend()

# Show the plot
plt.show()

In [ ]:
calc_metrics(encode_times)

In [ ]:
calc_metrics(decode_times)

# which tokens are matched

In [ ]:
# model

